In [ ]:
%matplotlib inline

# Write user defined Operators having third party dependencies {#ref_python_operators_with_deps}

This example shows how advanced DPF python plugins of Operators can be
created as standard python packages and how third party python modules
dependencies can be added to the package. For a first introduction on
user defined python Operators see example
`ref_wrapping_numpy_capabilities`{.interpreted-text role="ref"} and for
a simpler example on user defined python Operators as a package see
`ref_python_plugin_package`{.interpreted-text role="ref"}.

This plugin will hold an Operator which implementation depends on a
third party python module named [gltf](https://pypi.org/project/gltf/).
This Operator takes a path, a mesh and 3D vector field in input and
exports the mesh and the norm of the input field in a gltf file located
at the given path.


# Write Operator

For this more advanced use case, a python package is created. Each
Operator implementation derives from
`ansys.dpf.core.custom_operator.CustomOperatorBase`{.interpreted-text
role="class"} and a call to
:py`ansys.dpf.core.custom_operator.record_operator`{.interpreted-text
role="func"} records the Operators of the plugin. The complete package
looks like:


::: card
plugin

::: card
gltf_plugin

::: dropdown
\_\_init\_\_.py

::: literalinclude
plugins/gltf_plugin/\_\_init\_\_.py
:::
:::

::: dropdown
operators.py

::: literalinclude
plugins/gltf_plugin/operators.py
:::
:::

::: dropdown
operators_loader.py

::: literalinclude
plugins/gltf_plugin/operators_loader.py
:::
:::

::: dropdown
gltf_export.py

::: literalinclude
plugins/gltf_plugin/gltf_export.py
:::
:::

::: dropdown
texture.png

::: literalinclude
plugins/gltf_plugin/texture.png
:::
:::

::: dropdown
assets

`<plugins/gltf_plugin/assets/gltf_sites_winx64.zip>`{.interpreted-text
role="download"}

`<plugins/gltf_plugin/assets/gltf_sites_linx64.zip>`{.interpreted-text
role="download"}
:::
:::

::: dropdown
gltf_plugin.xml

::: {.literalinclude language="xml"}
plugins/gltf_plugin.xml
:::
:::
:::


# Load Plugin

Once a python plugin is written as a package, it can be loaded with the
function :py`ansys.dpf.core.core.load_library`{.interpreted-text
role="func"} taking as first argument the path to the directory of the
plugin, as second argument \"[py]()\" + any name identifying the plugin,
and as last argument the function\'s name exposed in the `__init__.py`
file and used to record operators.


In [ ]:
import os
from ansys.dpf import core as dpf
from ansys.dpf.core import examples

dpf.start_local_server()
dpf.load_library(
    os.path.join(os.getcwd(), "..", "..", "docs", "source", "examples", "07-python-operators", "plugins", "gltf_plugin"),
    "py_dpf_gltf",
    "load_operators")

Once the Plugin loaded, Operators recorded in the plugin can be used
with:


In [ ]:
new_operator = dpf.Operator("gltf_export")

This new Operator `gltf_export` requires a triangle surface mesh, a
displacement Field on this surface mesh as well as an export path as
inputs. To demo this new Operator, a
`ansys.dpf.core.model.Model`{.interpreted-text role="class"} on a simple
file is created,
`ansys.dpf.core.operators.mesh.tri_mesh_skin`{.interpreted-text
role="class"} Operator is used to extract the surface of the mesh in
triangles elements.


# Use the Custom Operator


In [ ]:
import tempfile
import os

model = dpf.Model(examples.static_rst)

mesh = model.metadata.meshed_region
skin_mesh = dpf.operators.mesh.tri_mesh_skin(mesh=mesh)

displacement = model.results.displacement()
displacement.inputs.mesh_scoping(skin_mesh)
displacement.inputs.mesh(skin_mesh)
new_operator.inputs.path(os.path.join(tempfile.mkdtemp(), "out"))
new_operator.inputs.mesh(skin_mesh)
new_operator.inputs.field(displacement.outputs.fields_container()[0])
new_operator.run()

print("operator ran successfully")

The gltf Operator output can be downloaded
`here<images/thumb/out02.glb>`{.interpreted-text role="download"}.
